In [2]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
# imports

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

In [3]:
# Load sequences and labels from disk
sequences = np.load('../sequences_augmented.npy')
labels = np.load('../labels_augmented.npy')


In [4]:
X = sequences
#output in one hot encoded format
y= to_categorical(labels).astype(int)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)    

In [6]:
from tensorflow import keras
from keras.layers import Dense, LSTM
from keras.models import Sequential
from keras.callbacks import TensorBoard

log_dir = os.path.join('../Logs')
tb_callback = TensorBoard(log_dir=log_dir)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# Reshape X_train to 2D array
num_samples, seq_length, num_features = X_train.shape
X_train_2d = np.reshape(X_train, (num_samples, seq_length * num_features))
X_test_2d = np.reshape(X_test, (X_test.shape[0], seq_length * num_features))

# Standardize the features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_2d)
X_test_scaled = scaler.transform(X_test_2d)


# Define parameter grid for grid search
param_grid = {'n_neighbors': [3, 5, 7, 9, 11], 'metric': ['euclidean', 'manhattan']}

# Initialize KNN classifier
knn = KNeighborsClassifier()

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=kf, scoring='accuracy')

# Perform grid search
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = grid_search.best_params_

In [9]:
print(best_params)

best_knn = grid_search.best_estimator_

print(best_knn)
# Train the best model
best_knn.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = best_knn.predict(X_test_scaled) 

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Best parameters:", best_params)
print("Accuracy:", accuracy)

{'metric': 'euclidean', 'n_neighbors': 3}
KNeighborsClassifier(metric='euclidean', n_neighbors=3)
Best parameters: {'metric': 'euclidean', 'n_neighbors': 3}
Accuracy: 0.5384615384615384


In [13]:
import joblib

# Save the best KNN model to a file
joblib.dump(best_knn, '../actions/best_knn_model.pkl')

['../actions/best_knn_model.pkl']

In [11]:
# Load the saved KNN model from file
loaded_knn_model = joblib.load('../actions/best_knn_model.pkl')


In [14]:
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix


y_pred_knn= best_knn.predict(X_test_scaled)
ytrue = np.argmax(y_test, axis=1).tolist()
ypred_knn = np.argmax(y_pred_knn, axis=1).tolist()

print(accuracy_score(ytrue, ypred_knn))
multilabel_confusion_matrix(ytrue, ypred_knn)

0.5446153846153846


array([[[416, 220],
        [  3,  11]],

       [[637,   3],
        [  7,   3]],

       [[629,   5],
        [ 11,   5]],

       [[636,   1],
        [  5,   8]],

       [[634,   3],
        [  7,   6]],

       [[639,   0],
        [  6,   5]],

       [[641,   2],
        [  3,   4]],

       [[640,   1],
        [  2,   7]],

       [[643,   1],
        [  4,   2]],

       [[645,   1],
        [  1,   3]],

       [[638,   1],
        [  4,   7]],

       [[641,   1],
        [  4,   4]],

       [[635,   7],
        [  6,   2]],

       [[638,   0],
        [  8,   4]],

       [[638,   1],
        [  4,   7]],

       [[638,   2],
        [  6,   4]],

       [[637,   2],
        [  6,   5]],

       [[639,   1],
        [  4,   6]],

       [[638,   1],
        [  7,   4]],

       [[645,   0],
        [  2,   3]],

       [[642,   1],
        [  4,   3]],

       [[640,   1],
        [  4,   5]],

       [[643,   0],
        [  3,   4]],

       [[640,   0],
        [  3, 